In [1]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from utils.tictoc import tic, toc

from src import constants as const

if const.dir.project() != os.getcwd():
    raise ValueError("Project directory not the same as in consts")

In [3]:
dataset_train_path = os.path.join(const.dir.data(), "df_adult_edited_train.csv")
dataset_test_path = os.path.join(const.dir.data(), "df_adult_edited_test.csv")

data_train = pd.read_csv(dataset_train_path)
data_test = pd.read_csv(dataset_test_path)
discrete_columns = data_train.columns[data_train.dtypes == "object"]

In [4]:
from sdv.tabular import CTGAN
?CTGAN

In [15]:
from sdv.tabular import TVAE
?TVAE

In [15]:
data_train

,age,workclass,fnlwgt,education,educational.num,marital.status,occupation,race,gender,capital.gain,capital.loss,hours.per.week,native.country,income
0,39,Government,77516,Bachelors,13,Never-married,Adm-clerical,White,Male,2174,0,40,US,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married,Exec-managerial,White,Male,0,0,13,US,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,White,Male,0,0,40,US,<=50K
3,53,Private,234721,<=12th,7,Married,Handlers-cleaners,Black,Male,0,0,40,US,<=50K
4,28,Private,338409,Bachelors,13,Married,Prof-specialty,Black,Female,0,0,40,Non-US,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,27,Private,257302,Assoc-acdm,12,Married,Tech-support,White,Female,0,0,38,US,<=50K
30158,40,Private,154374,HS-grad,9,Married,Machine-op-inspct,White,Male,0,0,40,US,>50K
30159,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,White,Female,0,0,40,US,<=50K
30160,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,White,Male,0,0,20,US,<=50K


In [13]:
from sdv.tabular import CopulaGAN

In [25]:
copgan = GaussianCopula()
copgan.fit(data_train)

C:\Users\Arne\anaconda3\envs\torch\lib\site-packages\scipy\stats\_continuous_distns.py:5350: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
C:\Users\Arne\anaconda3\envs\torch\lib\site-packages\scipy\stats\_distn_infrastructure.py:2640: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
C:\Users\Arne\anaconda3\envs\torch\lib\site-packages\scipy\stats\_continuous_distns.py:647: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
C:\Users\Arne\anaconda3\envs\torch\lib\site-packages\scipy\optimize\_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\Arne\anaconda3\envs\torch\lib\site-packages\copulas\univariate\truncated_gaussian.py:45: RuntimeWarning: invalid value encountered in double_scalars
  a = (self.min - loc) / sc

In [26]:
copgan.get_distributions()

{'age.value': 'copulas.univariate.truncated_gaussian.TruncatedGaussian',
 'workclass.value0': 'copulas.univariate.student_t.StudentTUnivariate',
 'workclass.value1': 'copulas.univariate.student_t.StudentTUnivariate',
 'workclass.value2': 'copulas.univariate.student_t.StudentTUnivariate',
 'workclass.value3': 'copulas.univariate.gaussian.GaussianUnivariate',
 'workclass.value4': 'copulas.univariate.gaussian.GaussianUnivariate',
 'fnlwgt.value': 'copulas.univariate.log_laplace.LogLaplace',
 'education.value0': 'copulas.univariate.student_t.StudentTUnivariate',
 'education.value1': 'copulas.univariate.gaussian.GaussianUnivariate',
 'education.value2': 'copulas.univariate.gaussian.GaussianUnivariate',
 'education.value3': 'copulas.univariate.log_laplace.LogLaplace',
 'education.value4': 'copulas.univariate.gaussian.GaussianUnivariate',
 'education.value5': 'copulas.univariate.gaussian.GaussianUnivariate',
 'education.value6': 'copulas.univariate.gaussian.GaussianUnivariate',
 'education.va

In [6]:
from sdv.tabular import GaussianCopula

In [7]:
train = pd.read_csv(os.path.join(const.dir.data_comparison(), "covtype_edited.csv"))
train

,Wilderness_Area,Soil_Type,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type
0,Wilderness_Area1,Soil_Type29,2596,51,3,258,0,510,221,232,148,6279,CoverType5
1,Wilderness_Area1,Soil_Type29,2590,56,2,212,-6,390,220,235,151,6225,CoverType5
2,Wilderness_Area1,Soil_Type12,2804,139,9,268,65,3180,234,238,135,6121,CoverType2
3,Wilderness_Area1,Soil_Type30,2785,155,18,242,118,3090,238,238,122,6211,CoverType2
4,Wilderness_Area1,Soil_Type29,2595,45,2,153,-1,391,220,234,150,6172,CoverType5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,Wilderness_Area3,Soil_Type2,2396,153,20,85,17,108,240,237,118,837,CoverType3
581008,Wilderness_Area3,Soil_Type2,2391,152,19,67,12,95,240,237,119,845,CoverType3
581009,Wilderness_Area3,Soil_Type2,2386,159,17,60,7,90,236,241,130,854,CoverType3
581010,Wilderness_Area3,Soil_Type2,2384,170,15,60,5,90,230,245,143,864,CoverType3


In [9]:
model = GaussianCopula()
model.fit(train)

C:\Users\Arne\anaconda3\envs\torch\lib\site-packages\scipy\stats\_continuous_distns.py:647: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
C:\Users\Arne\anaconda3\envs\torch\lib\site-packages\scipy\optimize\_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\Arne\anaconda3\envs\torch\lib\site-packages\scipy\stats\_continuous_distns.py:5350: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
C:\Users\Arne\anaconda3\envs\torch\lib\site-packages\scipy\stats\_distn_infrastructure.py:2640: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
C:\Users\Arne\anaconda3\envs\torch\lib\site-packages\scipy\optimize\_minpack_py.py:175: RuntimeWarning: The number of calls to function has reached maxfev = 600.
  warnings.warn(msg, Ru

In [22]:
model.sample()

,Wilderness_Area,Soil_Type,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type
0,Wilderness_Area1,Soil_Type29,3374,80,9,547,67,3002,162,214,152,4114,CoverType2
1,Wilderness_Area3,Soil_Type39,3284,172,10,387,62,1646,223,217,120,487,CoverType7
2,Wilderness_Area3,Soil_Type33,2919,360,3,206,73,1209,192,254,224,1926,CoverType2
3,Wilderness_Area1,Soil_Type29,2848,80,8,224,99,2801,164,232,170,1660,CoverType2
4,Wilderness_Area3,Soil_Type4,2846,325,14,164,-23,3050,186,215,173,1806,CoverType2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,Wilderness_Area1,Soil_Type29,2868,179,16,218,83,3449,211,188,109,3205,CoverType1
581008,Wilderness_Area1,Soil_Type29,2878,5,24,680,108,2539,223,197,100,1968,CoverType2
581009,Wilderness_Area1,Soil_Type30,3006,29,30,424,137,1761,215,191,100,1990,CoverType2
581010,Wilderness_Area1,Soil_Type10,3096,209,10,303,2,4156,208,235,185,5676,CoverType2


In [12]:
model.get_distributions()

{'Wilderness_Area.value0': 'copulas.univariate.gaussian.GaussianUnivariate',
 'Wilderness_Area.value1': 'copulas.univariate.gaussian.GaussianUnivariate',
 'Wilderness_Area.value2': 'copulas.univariate.gaussian.GaussianUnivariate',
 'Wilderness_Area.value3': 'copulas.univariate.gaussian.GaussianUnivariate',
 'Soil_Type.value0': 'copulas.univariate.gaussian.GaussianUnivariate',
 'Soil_Type.value1': 'copulas.univariate.gaussian.GaussianUnivariate',
 'Soil_Type.value2': 'copulas.univariate.gaussian.GaussianUnivariate',
 'Soil_Type.value3': 'copulas.univariate.gaussian.GaussianUnivariate',
 'Soil_Type.value4': 'copulas.univariate.gaussian.GaussianUnivariate',
 'Soil_Type.value5': 'copulas.univariate.gaussian.GaussianUnivariate',
 'Soil_Type.value6': 'copulas.univariate.gaussian.GaussianUnivariate',
 'Soil_Type.value7': 'copulas.univariate.log_laplace.LogLaplace',
 'Soil_Type.value8': 'copulas.univariate.gaussian.GaussianUnivariate',
 'Soil_Type.value9': 'copulas.univariate.gaussian.Gaussian

In [19]:
import ctgan
?ctgan.TVAESynthesizer.fit
# ?ctgan.CTGANSynthesizer

In [8]:
# tic()
# model = CTGAN(epochs=100, discriminator_steps=10, verbose=1, cuda=False)
# model.fit(data_train)
# toc()
print("Elapsed time: 2971.843488 seconds.")

Elapsed time: 2971.843488 seconds.


In [23]:
n_synthetic_datasets=10
overwrite=False
dir_gen_data_ctgan = os.path.join(const.dir.data_gen(), "CTGAN")
from IPython.display import clear_output

with tqdm(total=n_synthetic_datasets) as pbar:
    for i in range(n_synthetic_datasets):
        curr_path =  os.path.join(dir_gen_data_ctgan, f"gen{i}.csv")
        if overwrite or not os.path.exists(curr_path):
            model = CTGAN(epochs=100, batch_size=500, discriminator_steps=10, verbose=0, cuda=True,
                         embedding_dim=128, generator_dim=(256, 256), discriminator_dim=(256, 256),
                          pac=10, log_frequency=True
                         )
            model.fit(data_train)
            data = model.sample()
            data.to_csv(curr_path, index=False)
            clear_output()
        pbar.update(1)
        print(pbar)

100%|██████████| 10/10 [3:27:37<00:00, 1249.26s/it]


In [ ]:
dat = model.sample()

In [ ]:
dat.to_csv("temp_synthetic_data_cgan.csv", index=False)

In [9]:
?evaluate

In [8]:
from sdv.evaluation import evaluate
#evaluate(dat, data_train)

In [1]:
import ctgan